In [5]:
import pandas as pd
import sqlite3
conn = sqlite3.connect('db.sqlite')

tables = ["ANNOUNCEMENT", "COURSE", "DEPARTMENT", "STUDY_GROUP",
          "INSTRUCTOR", "JOIN_GROUP", "MEET", "OFFER_COURSE", "TAKE_COURSE", "IS_FRIEND_OF", "USER"]


for table in tables:
    print("Converting:", table)
    df = pd.read_csv(f"{table}.csv",)
    df.to_sql(table, conn, if_exists='replace', index=False)

Converting: ANNOUNCEMENT
Converting: COURSE
Converting: DEPARTMENT
Converting: STUDY_GROUP
Converting: INSTRUCTOR
Converting: JOIN_GROUP
Converting: MEET
Converting: OFFER_COURSE
Converting: TAKE_COURSE
Converting: IS_FRIEND_OF
Converting: USER
